In [15]:
import csv
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import xgboost as xgb

target = []
data = []

f = open(r'new_train.csv')
for line in f:        
    vec = line.split(",")    
    try:
        target.append(int(vec[-1]))
        data.append(vec[:-1])
    except:
        pass
f.close()

totaltreino = len(target)

print("Dados de treino: ", totaltreino)

f = open(r'test.csv')
for line in f:        
    vec = line.split(",")    
    try:        
        data.append(vec)
    except:
        pass
f.close()

data = np.array(data, dtype=np.float64)
    
print("Total de dados: ", len(data))

Dados de treino:  60020
Total de dados:  76020


In [16]:
def analiseColuna(matriz, col):
    valores = []
    contagem = []
    for i in range(len(matriz)):
        if not matriz[i][col] in valores: #novo valor
            valores.append(matriz[i][col])
            contagem.append(1)
        else: #valor ja existe, aumento a contagem
            contagem[valores.index(matriz[i][col])] += 1
    return valores,contagem

def colunasIguais(matriz, col1, col2):
    for i in range(len(matriz)):
        if(matriz[i][col1] != matriz[i][col2]):            
            return False
    return True

def colunaZerada(matriz, col):
    for i in range(len(matriz)):
        if(matriz[i][col] != 0.):            
            return False
    return True

def oneHot(rawData):
    array = np.array(rawData)
    keys = []
    for i in range(len(array.T)):
        conjunto = set(array.T[i])
        pos = list(conjunto)
        keys += pos

    dict_pos = {}
    for i in range(len(keys)):
        dict_pos[keys[i]] = i

    rawdata_tratado = []
    for i in range(len(rawData)):
        zeros = np.zeros(len(keys))
        for k in rawData[i]:
            zeros[dict_pos[k]] = 1
        rawdata_tratado.append(list(zeros))
    return np.array(rawdata_tratado)

def saveMatrixToFile(filename, matrix):
    with open(filename, "w") as f:
        writer = csv.writer(f, delimiter = ',', lineterminator='\n')
        writer.writerows(matrix)

In [17]:
i=0

while i < len(data[0]):
    j=0
    while j < len(data[0]):
        if i==j:
            j+=1
            continue
        if(colunaZerada(data, i)):
            data=np.delete(data, [i], axis=1)
            i-=1
            continue
        if(colunasIguais(data, i, j)):
            data=np.delete(data, [i], axis=1)            
            j-=1
        j+=1
    i+=1            

Colunas iguais em <b>data</b> foram removidas.

In [4]:
for i in range(len(data[0])):
    v,c = analiseColuna(data, i)
    if len(v) <= 30: #coluna pode sofrer one-hot-encoding
        coluna = data[:, i:i+1] #salvo uma cópia da coluna
        np.delete(data, [i], axis=1) #removo a coluna do dataset
        coluna = oneHot(coluna) #faço o one-hot-encoding da coluna
        data = np.concatenate((coluna, data), axis=1) #adiciono a coluna de volta, em one-hot, no dataset
print(len(data[0]))

925


Sabemos quais colunas possuem uma quantidade de valores razoáveis para sofrer one-hot-encoding.

In [19]:
data2 = np.array(data[:totaltreino])
target2 = np.array(target)

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

k = 10

print("Iniciando treino K-fold, k =", k)

X = data2
y = target2

kf = KFold(n_splits=k, shuffle = True)
print("Baseline: ", len(y[y==1])/float(len(y)) )
auc = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    score = roc_auc_score(y_pred, y_test)
    auc.append(score)
print("mean auc: ", np.mean(auc))

Iniciando treino K-fold, k = 10
Baseline:  0.03923692102632456
mean auc:  0.54746175414


In [24]:
import xgboost as xgb

dtrain = xgb.DMatrix(data2, target2)
param = {'max_depth':2, 'eta':1, 'silent':0, 'objective':'binary:logistic'}
res = xgb.cv(param, dtrain, metrics={'auc'}, nfold=10)

print(res)

   test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0       0.772041      0.010060        0.772522       0.001076
1       0.777533      0.011424        0.780812       0.009775
2       0.795760      0.012884        0.796964       0.004150
3       0.803087      0.014174        0.811104       0.004916
4       0.812954      0.013921        0.819313       0.004133
5       0.818477      0.014206        0.825514       0.003510
6       0.822619      0.013282        0.829571       0.003020
7       0.824736      0.013541        0.831877       0.002488
8       0.826116      0.013269        0.834306       0.001979
9       0.827581      0.013423        0.836427       0.001619
